In [1]:
# Inspiration
# Remove the column that only contains missing values
# Do men or women speed more often?
# Does gender affect who gets searched during a stop?
# During a search, how often is the driver frisked?
# Which year had the least number of stops?
# When a man is pulled over, how often is it for speeding?
# How does drug activity change by time of day?
# Do most stops occur at night?

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime,time, date
from dateutil import parser

In [22]:
police = pd.read_csv(r'C:\Users\godwi\GitHub\SQL\Police\police_project.csv')
police.head()

,stop_date,stop_time,county_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,1/2/2005,1:55,NaN,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
1,1/18/2005,8:15,NaN,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
2,1/23/2005,23:15,NaN,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
3,2/20/2005,17:15,NaN,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False
4,3/14/2005,10:00,NaN,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False


#  Checking the nature of the dataset

In [4]:
police.shape

(91741, 15)

In [5]:
police.info(verbose = False)  # check what data type is available

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91741 entries, 0 to 91740
Columns: 15 entries, stop_date to drugs_related_stop
dtypes: bool(2), float64(3), object(10)
memory usage: 9.3+ MB


In [6]:
# Total number of registered male and female drivers
police['driver_gender'].value_counts().sum()

86406

# Remove the column that only contains missing values

In [7]:
police.duplicated()== True

0        False
1        False
2        False
3        False
4        False
         ...  
91736    False
91737    False
91738    False
91739    False
91740    False
Length: 91741, dtype: bool

# Do men or women speed more often?

In [8]:
# ratio of male to female
police['driver_gender'].value_counts().to_frame()

,driver_gender
M,62895
F,23511


In [9]:
# % of male to female
round(police['driver_gender'].value_counts(1).to_frame(),3)
# Men speed more than women

,driver_gender
M,0.728
F,0.272


In [10]:
# check if there are any null values in driver_gender column ?
police.driver_gender.isnull().value_counts().to_frame()
# so we have 5,335 null values in the drive_gender column 

,driver_gender
False,86406
True,5335


# Does gender affect who gets searched during a stop?

In [11]:
police.loc[:, ['driver_gender', 'search_conducted']].groupby('driver_gender')\
.count().rename({'search_conducted' : 'Count_of_search'},axis = 1)

# Yes the male ate searched more than the female

,Count_of_search
driver_gender,
F,23511
M,62895


# During a search, how often is the driver frisked?

In [12]:
arrest = round(police.is_arrested.value_counts(100).to_frame(),2)
arrest
# Only 3% of the time , are drivers arrested

,is_arrested
False,0.97
True,0.03


# Which year had the least number of stops?

In [13]:
pd.DatetimeIndex(police.stop_date).year  # can use   ===  >  pd.DatetimeIndex(['stop_date']).year

Int64Index([2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005,
            ...
            2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015],
           dtype='int64', name='stop_date', length=91741)

In [14]:
least_year = police['stop_date'].to_frame()
least_year [:1]

,stop_date
0,1/2/2005


In [15]:
least_year['year'] = pd.DatetimeIndex(police['stop_date']).year

In [16]:
least_year.groupby('year').count().sort_values('stop_date', ascending = True).head(1)

,stop_date
year,
2005,2558


# When a man is pulled over, how often is it for speeding?

In [17]:
police[(police.violation.str.contains('Speeding')) & (police.driver_gender == 'M')]\
.violation.value_counts().to_frame().rename({'violation' :'no of violation by men'}, axis = 1)

,no of violation by men
Speeding,32979


In [18]:
police.loc[:, ['violation','driver_gender']].groupby ('driver_gender').get_group('M').value_counts()

violation            driver_gender
Speeding             M                32979
Moving violation     M                13020
Equipment            M                 8533
Other                M                 3627
Registration/plates  M                 2419
Seat belt            M                 2317
dtype: int64

# How does drug activity change by time of day?

In [19]:
tod_police = police [police['drugs_related_stop'] == True]\
.loc[:, ['stop_time', 'stop_duration' ,'drugs_related_stop']]
tod_police

,stop_time,stop_duration,drugs_related_stop
40,0:00,16-30 Min,True
41,0:00,30+ Min,True
118,1:20,30+ Min,True
139,0:20,16-30 Min,True
219,23:30,16-30 Min,True
...,...,...,...
91194,14:48,16-30 Min,True
91253,10:33,16-30 Min,True
91284,14:10,30+ Min,True
91672,11:05,16-30 Min,True


# Do most stops occur at night?

In [27]:
# first we have to group the time ofstops
# 2nd create a column and iterate with the time group
fn = police.loc[:, ['stop_time']]
fn.head()

,stop_time
0,1:55
1,8:15
2,23:15
3,17:15
4,10:00


In [29]:
fn['time_group'] = fn.apply(lambda x : x.stop_time[:2], axis = 1)
fn.head()

,stop_time,time_group
0,1:55,1:
1,8:15,8:
2,23:15,23
3,17:15,17
4,10:00,10


In [30]:
fn['group'] = fn.apply(lambda x: 'AM' if x.name in ['time_group'] > 0 <12 else 'DAY' \
                       if x.name in ['time_group']> 12 <19 else 'Night', axis = 1)

In [31]:
fn.drop('group', axis = 1, inplace = True)

In [32]:
fn['new'] = [ 'AM' if  0 < x < 12  else 'PM' if 12 < x < 19 else 'Night' for x in fn.time_group]

TypeError: '<' not supported between instances of 'int' and 'str'

In [ ]:
fn.new.value_counts().to_frame()
# No most stops occur during the day

In [ ]:
pip install pgcsv

In [ ]:
pip install SQLAlchemy

In [ ]:
pip install psycopg2

In [33]:
import sqlalchemy